In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

24/05/06 12:25:10 WARN Utils: Your hostname, aliyu-XPS-15-9550 resolves to a loopback address: 127.0.1.1; using 192.168.199.55 instead (on interface wlp2s0)
24/05/06 12:25:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 12:25:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/06 12:25:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/06 12:25:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/06 12:25:12 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/05/06 12:25:12 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/05/06 12:25:12 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attemptin

In [4]:
data_path = './Data'

In [5]:
json_df2_path = data_path + '/utilization.json'
df_util = spark.read.format('json').load(json_df2_path)

In [6]:
df_util.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [7]:
df_util.createOrReplaceTempView('utilization')

In [8]:
df_util.count()

500000

In [9]:
df_util.describe().show()

24/05/06 12:25:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------------+-------------------+-------------------+------------------+------------------+
|summary|    cpu_utilization|     event_datetime|        free_memory|         server_id|     session_count|
+-------+-------------------+-------------------+-------------------+------------------+------------------+
|  count|             500000|             500000|             500000|            500000|            500000|
|   mean| 0.6205177399999957|               NULL|0.37912809999999864|             124.5|          69.59616|
| stddev|0.15875173872912945|               NULL|0.15830931278376276|14.430884120553191|14.850676696352851|
|    min|               0.22|03/05/2019 08:06:14|                0.0|               100|                32|
|    max|                1.0|04/09/2019 01:22:46|               0.78|               149|               105|
+-------+-------------------+-------------------+-------------------+------------------+------------------+



In [10]:
df_util.stat.corr('cpu_utilization','free_memory')

-0.47047715730807554

In [11]:
df_util.stat.corr('session_count','free_memory')

-0.5008320848876574

In [12]:
df_util.stat.freqItems(('server_id','session_count')).show()

+--------------------+-----------------------+
| server_id_freqItems|session_count_freqItems|
+--------------------+-----------------------+
|[137, 146, 101, 1...|   [92, 101, 83, 104...|
+--------------------+-----------------------+



In [13]:
df_util.sample(fraction=0.05, withReplacement=False)

DataFrame[cpu_utilization: double, event_datetime: string, free_memory: double, server_id: bigint, session_count: bigint]

In [14]:
df_util_sample = df_util.sample(fraction=0.05, withReplacement=False)

In [15]:
df_util_sample.count()

25086

In [17]:
spark.sql('SELECT min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) FROM utilization').show()

+--------------------+--------------------+-----------------------+
|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+--------------------+--------------------+-----------------------+
|                0.22|                 1.0|    0.15875173872912945|
+--------------------+--------------------+-----------------------+



In [22]:
spark.sql('SELECT min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
                                FROM utilization \
                                GROUP BY server_id').show()

+--------------------+--------------------+-----------------------+
|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+--------------------+--------------------+-----------------------+
|                0.56|                0.96|    0.11617507884178278|
|                0.51|                0.91|    0.11521679513850511|
|                0.22|                0.62|    0.11531539914568226|
|                0.27|                0.67|     0.1152264191787964|
|                0.29|                0.69|    0.11510721467869486|
|                 0.6|                 1.0|    0.11651726263197697|
|                0.56|                0.96|    0.11549678751286807|
|                0.52|                0.92|    0.11528867845082576|
|                0.58|                0.98|    0.11544345150353687|
|                0.35|                0.75|    0.11533251724450215|
|                0.45|                0.85|    0.11597417369783877|
|                0.36|                0.76|    0

In [23]:
spark.sql('SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
                                FROM utilization \
                                GROUP BY server_id').show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      103|                0.56|                0.96|    0.11617507884178278|
|      104|                0.51|                0.91|    0.11521679513850511|
|      106|                0.22|                0.62|    0.11531539914568226|
|      100|                0.27|                0.67|     0.1152264191787964|
|      105|                0.29|                0.69|    0.11510721467869486|
|      101|                 0.6|                 1.0|    0.11651726263197697|
|      102|                0.56|                0.96|    0.11549678751286807|
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      110|                0.35|                0.75|    0.11533

In [24]:
spark.sql('SELECT server_id, FLOOR(cpu_utilization*100/10) bucket FROM utilization').show()

+---------+------+
|server_id|bucket|
+---------+------+
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     4|
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     6|
|      100|     6|
|      100|     5|
|      100|     2|
|      100|     4|
|      100|     4|
|      100|     6|
|      100|     4|
|      100|     5|
+---------+------+
only showing top 20 rows



In [25]:
spark.sql('SELECT count(*), FLOOR(cpu_utilization*100/10) bucket FROM utilization GROUP BY bucket ORDER BY bucket').show()

+--------+------+
|count(1)|bucket|
+--------+------+
|    8186|     2|
|   37029|     3|
|   68046|     4|
|  104910|     5|
|  116725|     6|
|   88242|     7|
|   56598|     8|
|   20207|     9|
|      57|    10|
+--------+------+

